In [ ]:
!pip install deap


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 135.4/135.4 kB 2.8 MB/s eta 0:00:00


In [ ]:
from deap import base, creator, tools, gp


In [ ]:
import operator
import random
import numpy as np
from deap import base, creator, tools, gp

In [ ]:
# Definir las funciones lógicas
def AND(a, b): return a and b
def OR(a, b): return a or b
def NOT(a): return not a

In [ ]:
# Conjunto de funciones
pset = gp.PrimitiveSet("MAIN", 4)  # 4 entradas: A, B, C, D
pset.addPrimitive(AND, 2)
pset.addPrimitive(OR, 2)
pset.addPrimitive(NOT, 1)

In [ ]:
# Nombres de las entradas
pset.renameArguments(ARG0="A", ARG1="B", ARG2="C", ARG3="D")

# Crear el problema
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", gp.PrimitiveTree, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("expr", gp.genFull, pset=pset, min_=1, max_=3)
toolbox.register("individual", tools.initIterate, creator.Individual, toolbox.expr)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)


/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'FitnessMax' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "
/usr/local/lib/python3.10/dist-packages/deap/creator.py:185: RuntimeWarning: A class named 'Individual' has already been created and it will be overwritten. Consider deleting previous creation of that class or rename it.
  warnings.warn("A class named '{0}' has already been created and it "


In [ ]:
# Evaluación de aptitud
def evaluate(individual):
    func = toolbox.compile(expr=individual)
    # Tabla de verdad para un codificador de 7 segmentos
    truth_table = [
        ([0, 0, 0, 0], [1, 1, 1, 1, 1, 1, 0]),  # Representa "0"
        ([0, 0, 0, 1], [0, 1, 1, 0, 0, 0, 0]),  # Representa "1"
        # Agrega las demás entradas y salidas para "2", "3", ..., "9"
    ]
    score = 0
    for inputs, expected_output in truth_table:
        output = [func(*inputs) for _ in range(7)]  # Simula las salidas
        score += sum(1 for o, e in zip(output, expected_output) if o == e)
    return score,

In [ ]:
# Registro de herramientas
toolbox.register("compile", gp.compile, pset=pset)
toolbox.register("evaluate", evaluate)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("mate", gp.cxOnePoint)
toolbox.register("mutate", gp.mutUniform, expr=toolbox.expr, pset=pset)


In [ ]:
# Configuración del algoritmo
toolbox.register("map", map)

def main():
    random.seed(42)
    population = toolbox.population(n=100)
    hof = tools.HallOfFame(1)
    stats = tools.Statistics(lambda ind: ind.fitness.values)
    stats.register("avg", np.mean)
    stats.register("max", np.max)

In [ ]:
import random
import numpy as np
from deap import base, creator, tools, algorithms

# Crear los tipos básicos
creator.create("FitnessMin", base.Fitness, weights=(-1.0,))
creator.create("Individual", list, fitness=creator.FitnessMin)

# Crear toolbox
toolbox = base.Toolbox()
toolbox.register("attr_float", random.uniform, -10, 10)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_float, n=5)
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def evaluate(individual):
    return sum(x**2 for x in individual),  # Evaluar la suma de cuadrados

toolbox.register("mate", tools.cxBlend, alpha=0.5)
toolbox.register("mutate", tools.mutGaussian, mu=0, sigma=1, indpb=0.2)
toolbox.register("select", tools.selTournament, tournsize=3)
toolbox.register("evaluate", evaluate)

# Configurar estadísticas y Hall of Fame
stats = tools.Statistics(key=lambda ind: ind.fitness.values)
stats.register("min", np.min)
stats.register("avg", np.mean)

hof = tools.HallOfFame(1)

def main():
    random.seed(42)
    population = toolbox.population(n=50)

    # Algoritmo evolutivo
    population, logbook = algorithms.eaSimple(population, toolbox, cxpb=0.5, mutpb=0.2,
                                              ngen=40, stats=stats, halloffame=hof,
                                              verbose=True)

    # Imprimir resultados
    print("Mejor individuo:", hof[0])
    print("Aptitud:", hof[0].fitness.values[0])

if __name__ == "__main__":
    main()


gen	nevals	min    	avg    
0  	50    	78.1453	170.877
1  	34    	57.4474	131.565
2  	30    	24.9891	93.9177
3  	29    	14.9678	69.5816
4  	33    	14.9678	50.6336
5  	27    	4.4407 	33.7353
6  	28    	4.4407 	22.863 
7  	30    	4.80102	16.0411
8  	34    	1.59264	11.5087
9  	26    	0.319029	8.29523
10 	29    	1.44209 	5.26378
11 	29    	1.45261 	3.56065
12 	30    	0.247436	2.26804
13 	28    	0.247436	2.11053
14 	24    	0.133757	1.24246
15 	35    	0.133757	0.805258
16 	31    	0.133757	0.635552
17 	33    	0.0611178	0.704081
18 	36    	0.0504591	0.401196
19 	28    	0.0238136	0.266672
20 	32    	0.0351805	0.448557
21 	27    	0.0342132	0.323325
22 	26    	0.0176376	0.158408
23 	23    	0.0162658	0.082841
24 	22    	0.00363994	0.0832444
25 	30    	0.00363994	0.139067 
26 	29    	0.00348742	0.149255 
27 	35    	0.000510733	0.230086 
28 	26    	0.000510733	0.395897 
29 	32    	0.000510733	0.420255 
30 	30    	0.000510733	0.221244 
31 	33    	0.000137926	0.195089 
32 	35    	0.000137926	0.138669 
